<a href="https://colab.research.google.com/github/lphuong304/CS114.L21/blob/main/COLAB_ASSIGNMENTS/COLAB_ASSIGNMENT_09_06/sarcasm_article.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center"><b>BÀI TẬP COLAB SỐ 04 - 07/06/2021</b></h1>
<h1 align="center"><b>CRAWLING HEALINE FROM NEWS WEBSITES</b></h1>

---

###LỚP: CS114.L21
###NHÓM:
1. Nguyễn Ngọc Lan Phương - *19520227*
2. Hoàng Anh Tú - *19522450*
3. Nguyễn Quốc Huy - *19521623*

Nguồn tham khảo:
* [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)



* Link các trang web:
  + Trang web tin tức châm biếm:
    + [Clickhole](https://clickhole.com/)
    + [The Babylon Bee](https://babylonbee.com/)
    + [Daily Bonet](https://dailybonnet.com/)
  + Trang web tin tức chính thống:
    + [Clickhole](https://clickhole.com/)
    + [The Babylon Bee](https://babylonbee.com/)
    + [Daily Bonet](https://dailybonnet.com/)



References:
1. [Import Packages](#part1)
2. [Crawling from Sasatirical News Websites](#sasatirical)
3. [Crawling from Official News Websites](#news)
4. [Nhận xét](#nhanxet)

#1. Import Packages
<a name="part1"></a>

In [2]:
!pip install requests


In [3]:
!pip install b4

     |████████████████████████████████| 71kB 3.4MB/s 
     |████████████████████████████████| 61kB 4.3MB/s 
     |████████████████████████████████| 245kB 7.1MB/s 
     |████████████████████████████████| 61kB 4.6MB/s 
     |████████████████████████████████| 962kB 7.3MB/s 
  Created wheel for dkimpy: filename=dkimpy-1.0.5-cp37-none-any.whl size=48749 sha256=b52fa538fe6bd8edf48a252412025a833196870576e54d0f29518034035c1f77
  Stored in directory: /root/.cache/pip/wheels/d6/2a/98/c08efef1abdf8ad07b3cdffdd7eefb0c14afc0769234c17221
Successfully built dkimpy
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [4]:
!pip install newspaper3k

     |████████████████████████████████| 215kB 4.3MB/s 
     |████████████████████████████████| 81kB 3.9MB/s 
     |████████████████████████████████| 92kB 4.9MB/s 
     |████████████████████████████████| 7.4MB 6.1MB/s 
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp37-none-any.whl size=3358 sha256=560fbd9fafc2dfa3cf5fa51ba43918dcdfd050d0722ea56f6fedd07e365756e2
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp37-none-any.whl size=7398404 sha256=dc4b9a8ef4ce865c2bf8230a56d94dbf2f3701b86dddeb5c1343e01280c19b1e
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35da20627c013f438ed34
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-cp37-none-any.whl size=13553 sha256=de4e7985e8e052d7c5e3e4dc120b375ec2c40ec546bf0f327c563f9b9ea3c095
  Stored in directory: /root/.cache/pip/wheels/81/2b/43/a02ede72324dd40cdd7ca53aad718c771

In [5]:
!pip  install fsspec

     |████████████████████████████████| 122kB 4.3MB/s 


In [6]:
import requests
from bs4 import BeautifulSoup as soup
import json
import pandas as pd
from io import StringIO
import re
from datetime import datetime as dt


# 2. Crawling from Sasatirical News Websites
<a name="sasatirical"></a>


##Crawling from https://babylonbee.com/news

In [7]:
content1 = []
res1=[]
n_pages1 = 255 #chúng em có lướt thử các trang và nhận thấy trang 250 đã là năm 2018
for i in range(1, n_pages1):
  url_1 = "https://babylonbee.com/news?page=" + str(i)
  site_1 = requests.get(url_1)
  page_soup_1 = soup(site_1.content,'html.parser')
  page_soup_1 = page_soup_1.find_all("article-card" ) 
  #print(page_soup_1)
  for i in page_soup_1:
    #chuyển đổi dòng html sang dạng chuỗi và biến đổi thuận tiện cho việc cắt chuỗi, lấy các thông tin cần thiết
      i = str(i).replace("</article-card>","").replace("&quot;","").replace("'>", ">") 

      #find string express path link
      index1 = i.find(":path") + 7 
      index2 = i.find(r':primary_category') - 2
      link_article_1 = (i[index1: index2])
      
      #find string express path link of date
      index3 = i.find(":published_on") + 15
      index4 = i.find(":title") - 2
      time1 = (i[index3:index4]) 

      #find headline
      index5 = i.find(":title=") + 8
      index6 = i.find(r'>') - 1
      headtitle_1 = i[index5:index6].replace('\"','')
      is_sacarsm_1 = '1'

      content1.append({'article_link':"https://babylonbee.com/"+link_article_1, 'headline':headtitle_1, 'date': time1, "is_sarcastic":1})

json_res_1 = json.dumps(content1)
print(json_res_1)

[{"article_link": "https://babylonbee.com/'/news/new-york-times-now-claims-1619-project-was-satire'", "headline": "New York Times Now Claims '1619 Project' Was Satire", "date": "'June 9th, 2021'", "is_sarcastic": 1}, {"article_link": "https://babylonbee.com/'/news/10-reasons-homeschooling-is-extremely-dangerous'", "headline": "10 Most Devastating Impacts Of Homeschooling Your Kids", "date": "'June 9th, 2021'", "is_sarcastic": 1}, {"article_link": "https://babylonbee.com/'/news/i-do-not-study-science-i-am-the-science-growls-grizzled-bald-dr-fauci-to-reporters'", "headline": "'I Don't Study Science, I AM The Science,' Growls Grizzled, Bald Dr. Fauci To Reporters", "date": "'June 9th, 2021'", "is_sarcastic": 1}, {"article_link": "https://babylonbee.com/'/news/half-black-man-ordered-to-pay-himself-reparations'", "headline": "Half-Black Man Ordered To Pay Himself Reparations", "date": "'June 9th, 2021'", "is_sarcastic": 1}, {"article_link": "https://babylonbee.com/'/news/experts-unsure-wher

In [8]:
df1 = pd.read_json(StringIO(json_res_1))
mask = (df1['date'] >= '2018-01-01') #chỉ lấy những bài viết trong 3 năm kể từ đây
df1 = df1.loc[mask]
df1

,article_link,headline,date,is_sarcastic
0,https://babylonbee.com/'/news/new-york-times-n...,New York Times Now Claims '1619 Project' Was S...,2021-06-09,1
1,https://babylonbee.com/'/news/10-reasons-homes...,10 Most Devastating Impacts Of Homeschooling Y...,2021-06-09,1
2,https://babylonbee.com/'/news/i-do-not-study-s...,"'I Don't Study Science, I AM The Science,' Gro...",2021-06-09,1
3,https://babylonbee.com/'/news/half-black-man-o...,Half-Black Man Ordered To Pay Himself Reparations,2021-06-09,1
4,https://babylonbee.com/'/news/experts-unsure-w...,Experts Completely Baffled As To Where Joe Bid...,2021-06-09,1
...,...,...,...,...
5075,https://babylonbee.com/'/news/local-church-loo...,Local Church Looks To Attract Youth With 'Tith...,2018-07-13,1
5076,https://babylonbee.com/'/news/liberal-argues-r...,Liberal Argues Rules Of Basketball Are Flawed ...,2018-07-13,1
5077,https://babylonbee.com/'/news/triumph-this-man...,Triumph: This Man Whittled His Friends List Do...,2018-07-13,1
5078,https://babylonbee.com/'/news/the-bee-explains...,The Bee Explains: Democratic Socialism,2018-07-13,1


In [9]:
df1.to_csv("/content/test1.csv")

##Crawling from https://clickhole.com/

In [10]:
n_pages2 = 1176 #trang clickhole có tổng tất cả 1176 trang.
content2 = []
for i in range(1,n_pages2):
  url_2 = "https://clickhole.com/page/" + str(i) + "/"
  site_2 = requests.get(url_2)
  page_soup_2 = soup(site_2.content,'html.parser')
  page_soup_2 = page_soup_2.find_all("div",{"class":"post-header"} )
  for i in page_soup_2:
      link_article_2 = i.find("a")['href']
      headtitle_2 = i.find("h2", {"class": "post-title"}).get_text()
      time2 = i.find("div", {"class": "post-byline"}).get_text()
      headtitle_2 = headtitle_2.replace("\n","").replace(time2,"").replace("\u2019","")
      content2.append({'article_link':link_article_2, 'headline':headtitle_2,'date':time2, "is_sarcastic":1})
json_res_2 = json.dumps(content2)
print(json_res_2)
    

[{"article_link": "https://clickhole.com/category/lifestyle/", "headline": "Tragic: Your Heartfelt Facebook Post About Your Friends Death Isnt Getting The Engagement It Would Have In 2013", "date": "June 9, 2021", "is_sarcastic": 1}, {"article_link": "https://clickhole.com/category/lifestyle/", "headline": "After 93 Years On Earth, Thats All The Wisdom He Has To Offer? Grandpa Just Said Its Important To Have Fun In Life", "date": "June 9, 2021", "is_sarcastic": 1}, {"article_link": "https://clickhole.com/category/news/", "headline": "Drunk With Power: This Crossing Guard Is Attempting To Stop An Ambulance Thats Going 70 With Its Siren Wailing And Everything", "date": "June 8, 2021", "is_sarcastic": 1}, {"article_link": "https://clickhole.com/category/lifestyle/", "headline": "Someone At Work Mustve Died Or Something: Mom Just Said During Dinner That You Never Know When Your Time Will Come, Super Unprompted", "date": "June 8, 2021", "is_sarcastic": 1}, {"article_link": "https://clickhol

In [11]:
df2 = pd.read_json(StringIO(json_res_2))
mask2 = (df2['date'] >= '2018-01-01') #chỉ lấy những bài viết trong 3 năm kể từ đây
df2 = df2.loc[mask2]
df2

,article_link,headline,date,is_sarcastic
0,https://clickhole.com/category/lifestyle/,Tragic: Your Heartfelt Facebook Post About You...,2021-06-09,1
1,https://clickhole.com/category/lifestyle/,"After 93 Years On Earth, Thats All The Wisdom ...",2021-06-09,1
2,https://clickhole.com/category/news/,Drunk With Power: This Crossing Guard Is Attem...,2021-06-08,1
3,https://clickhole.com/category/lifestyle/,Someone At Work Mustve Died Or Something: Mom ...,2021-06-08,1
4,https://clickhole.com/category/lifestyle/,Whats Their Problem? All The Cloven Hoofed Ani...,2021-06-07,1
...,...,...,...,...
2513,https://clickhole.com/category/lifestyle/,5 Times John Madden Filled The Silence During ...,2018-01-08,1
2514,https://clickhole.com/category/news/,Good On Them: The Black Eyed Peas Have Announc...,2018-01-08,1
2515,https://clickhole.com/category/lifestyle/,Making A Difference: This Nonprofit Collects T...,2018-01-08,1
2516,https://clickhole.com/category/lifestyle/,Wedding Bells! Two People Who Bullied You In H...,2018-01-08,1


In [12]:
df2.to_csv("/content/test2.csv")

##Crawling from https://dailybonnet.com/

In [13]:
#vì link bài báo trên web này có được phân loại chung theo format "https://dailybonnet.com/{year}/{month}/page/"
#khi lướt qua các bài báo được search theo kiểu ngày tháng trên, thì chúng em nhận thấy mỗi tháng như vậy có trung bình 30 - 35 bài viết
#chia thành 5 bài mỗi page
#nên chúng em nghĩ đến hướng cho vòng lặp chạy theo năm (2019->2021: 12 tháng, 6 pages, 5 bài báo/page)
#riêng năm 2021 chỉ tính đến các bài báo tháng 5
content3=[]
for y in range(2018,2022): 
  if y == 2021:
    for m in range(1,6):
        for i in range(1,8):
          url_3 = "https://dailybonnet.com/" + str(y) + "/" + str(m) + "/page/" + str(i) + "/" 
          site_3 = requests.get(url_3)
          page_soup_3 = soup(site_3.content,'html.parser')
          page_soup_3 = page_soup_3.find_all("header",{"class":"mh-posts-list-header"})
          #print(page_soup_3)
          for j in page_soup_3:
            link_article_3 = j.find("a")["href"]
            headtitle_3 = j.find("a")["title"]
            headtitle_3 = headtitle_3.replace("\n","").replace("\u2019","").replace("\u2018","")
            get_time = j.find("span",{"class":"entry-meta-date updated"}).get_text()
            is_sacarsm = '1'
            content3.append({'article_link':link_article_3, 'headline':headtitle_3,"date" : get_time, "is_sarcastic":1})
  else:
    for m in range(1,13):
      for i in range(1,8):
          url_3 = "https://dailybonnet.com/" + str(y) + "/" + str(m) + "/page/" + str(i) + "/" 
          site_3 = requests.get(url_3)
          page_soup_3 = soup(site_3.content,'html.parser')
          page_soup_3 = page_soup_3.find_all("header",{"class":"mh-posts-list-header"})
          #print(page_soup_3)
          for j in page_soup_3:
            link_article_3 = j.find("a")["href"]
            headtitle_3 = j.find("a")["title"]
            headtitle_3 = headtitle_3.replace("\n","").replace("\u2019","").replace("\u2018","")
            get_time = j.find("span",{"class":"entry-meta-date updated"}).get_text()
            is_sacarsm = '1'
            content3.append({'article_link':link_article_3, 'headline':headtitle_3,"date" : get_time, "is_sarcastic":1})
json_res_3 = json.dumps(content3)
print(json_res_3)

[{"article_link": "https://dailybonnet.com/many-typical-mennonite-things-done/", "headline": "How Many of These Typical Mennonite Things Have You Done?", "date": "January 31, 2018", "is_sarcastic": 1}, {"article_link": "https://dailybonnet.com/mennonite-town-bans-use-turn-signals/", "headline": "Mennonite Town Bans Using Turn Signals", "date": "January 30, 2018", "is_sarcastic": 1}, {"article_link": "https://dailybonnet.com/police-raid-mennonite-gambling-den-seize-hundreds-rook-cards/", "headline": "Police Raid Mennonite Gambling Den, Seize Hundreds of Rook Cards", "date": "January 29, 2018", "is_sarcastic": 1}, {"article_link": "https://dailybonnet.com/rare-plautdietsch-dictionary-nets-2-3-million-sothebys-auction/", "headline": "Rare Plautdietsch Dictionary Nets $2.3 Million at Sothebys Auction", "date": "January 29, 2018", "is_sarcastic": 1}, {"article_link": "https://dailybonnet.com/millennial-puts-nicest-ripped-jeans-church-sunday/", "headline": "Millennial Puts on Her Nicest Ripp

In [14]:
df3 = pd.read_json(StringIO(json_res_3))
df3

,article_link,headline,date,is_sarcastic
0,https://dailybonnet.com/many-typical-mennonite...,How Many of These Typical Mennonite Things Hav...,2018-01-31,1
1,https://dailybonnet.com/mennonite-town-bans-us...,Mennonite Town Bans Using Turn Signals,2018-01-30,1
2,https://dailybonnet.com/police-raid-mennonite-...,"Police Raid Mennonite Gambling Den, Seize Hund...",2018-01-29,1
3,https://dailybonnet.com/rare-plautdietsch-dict...,Rare Plautdietsch Dictionary Nets $2.3 Million...,2018-01-29,1
4,https://dailybonnet.com/millennial-puts-nicest...,Millennial Puts on Her Nicest Ripped Jeans for...,2018-01-28,1
...,...,...,...,...
1320,https://dailybonnet.com/kitchener-man-finally-...,Kitchener Man Finally Meets His Waterloo,2021-05-05,1
1321,https://dailybonnet.com/mennonite-authors-eage...,Mennonite Authors Eager to Get their Books Ban...,2021-05-04,1
1322,https://dailybonnet.com/government-now-requiri...,Government Now Requiring Masks to Fully Cover ...,2021-05-03,1
1323,https://dailybonnet.com/millions-flee-communis...,Millions Flee “Communist” Canada for a Better ...,2021-05-02,1


In [15]:
df3.to_csv("/content/test3.csv")

Tạo một dataframe chỉ gồm các bài báo tin tức châm biếm gộp từ 3 trang em vừa kiếm tính từ đầu năm 2019 đến đây

In [16]:
frames = [df1, df2, df3]

result = pd.concat(frames)
result = result.reset_index()
result

,index,article_link,headline,date,is_sarcastic
0,0,https://babylonbee.com/'/news/new-york-times-n...,New York Times Now Claims '1619 Project' Was S...,2021-06-09,1
1,1,https://babylonbee.com/'/news/10-reasons-homes...,10 Most Devastating Impacts Of Homeschooling Y...,2021-06-09,1
2,2,https://babylonbee.com/'/news/i-do-not-study-s...,"'I Don't Study Science, I AM The Science,' Gro...",2021-06-09,1
3,3,https://babylonbee.com/'/news/half-black-man-o...,Half-Black Man Ordered To Pay Himself Reparations,2021-06-09,1
4,4,https://babylonbee.com/'/news/experts-unsure-w...,Experts Completely Baffled As To Where Joe Bid...,2021-06-09,1
...,...,...,...,...,...
8918,1320,https://dailybonnet.com/kitchener-man-finally-...,Kitchener Man Finally Meets His Waterloo,2021-05-05,1
8919,1321,https://dailybonnet.com/mennonite-authors-eage...,Mennonite Authors Eager to Get their Books Ban...,2021-05-04,1
8920,1322,https://dailybonnet.com/government-now-requiri...,Government Now Requiring Masks to Fully Cover ...,2021-05-03,1
8921,1323,https://dailybonnet.com/millions-flee-communis...,Millions Flee “Communist” Canada for a Better ...,2021-05-02,1


# 3. Crawling from Official News Websites
<a name="news"></a>

##Crawling from https://www.cbsnews.com/latest/politics/

In [17]:
# section = ['us', 'world', 'politics', 'entertainment', 'health', 'moneywatch','technology','science','crime','sports' ]
content4 = []
# for sec in section:
for i in range(2, 1872):
  url4 = "https://www.cbsnews.com/latest/politics/" +  str(i) + "/"
  #print(url)
  reqs = requests.get(url4)
  page_source4 = soup(reqs.content, 'html.parser')
  page_source4 = page_source4.find_all("a",{"class":"item__anchor"})
  #print(page_source)
  for line in page_source4:
    headtitle4 = line.find('h4', class_ ='item__hed ').get_text()
    headtitle4 = headtitle4.replace("\n","") # bỏ /n trong việc xuất chuỗi ký tự
    #print(headtitle)
    link4 = line.get('href')
    #print(link)
    time4 = line.find('li', class_="item__date").get_text()
    if len(time4) != 10:
      time4 = time4 + ', 2021' 
    #print(time)
    if 'ago' not in time4:
      content4.append({'article_link':link4, 'headline':headtitle4, 'date': time4, "is_sarcastic":0})
json_res4 = json.dumps(content4)
#print(json_res)

In [18]:
df4 = pd.read_json(StringIO(json_res4))
mask4 = (df4['date'] >= '2018-01-01') #chỉ lấy những bài viết trong 3 năm kể từ đây
df4 = df4.loc[mask4]
df4

,article_link,headline,date,is_sarcastic
0,https://www.cbsnews.com/news/capitol-police-ri...,Senate report finds sweeping failu...,2021-06-08,0
1,https://www.cbsnews.com/news/house-republicans...,GOP lawmakers ask CDC to probe whe...,2021-06-08,0
2,https://www.cbsnews.com/news/colonial-pipeline...,Colonial Pipeline CEO defends hand...,2021-06-08,0
3,https://www.cbsnews.com/news/united-states-sen...,Senate confirms Biden's first judg...,2021-06-08,0
4,https://www.cbsnews.com/news/equality-act-lgbt...,Senate could expand LGBTQ protecti...,2021-06-08,0
...,...,...,...,...
43004,https://www.cbsnews.com/pictures/ted-bundy-ser...,Ted Bundy: The serial killer's fin...,2021-06-05,0
43005,https://www.cbsnews.com/pictures/whats-coming-...,"New on Netflix, Amazon, Disney+, H...",2021-05-27,0
43006,https://www.cbsnews.com/pictures/richest-peopl...,Richest people in the world,2021-05-26,0
43007,https://www.cbsnews.com/pictures/richard-ramir...,"Richard Ramirez: The story, the ev...",2021-05-21,0


In [19]:
df3.to_csv("/content/test4.csv")

## Crawling from https://fortune.com/

In [20]:
file_out5 = []
for i in range(195): 
  url5 = 'https://fortune.com/section/politics/page/'+str(i)+'/'
  reqs5 = requests.get(url5)
  page_source5 = soup(reqs5.content, 'html.parser')
  page_source5 = page_source5.find_all("li",{"class":"termArchiveContentList__item--1LvxK"})
  #print(page_source)
  for line in page_source5:
    headtitles5 = line.find("div", class_ = 'termArchiveContentListItem__title--14jcP')
    times5 = line.find("div", class_ = "termArchiveContentListItem__meta--3RUO6")
    headtitle5 = headtitles5.find('a', class_ = '')
    #print(headtitle)
    link = headtitle5.get('href')
    title = headtitle5.get_text()
    time = times5.find('p').get_text()
    # print(title)
    # print(link)
    # print(time)
    file_out5.append({'article_link':link, 'headline':title, 'date': time, "is_sarcastic":0})
json_res = json.dumps(file_out5)
df5 = pd.read_json(StringIO(json_res))
df5
  


,article_link,headline,date,is_sarcastic
0,https://fortune.com/2021/06/09/childcare-crisi...,"With the childcare crisis front and center, th...",2021-06-09,0
1,https://fortune.com/2021/06/08/us-senate-bill-...,The Senate is looking to pass legislation that...,2021-06-08,0
2,https://fortune.com/2021/06/08/ecb-monetary-po...,Why inflation could end the truce at Europe’s ...,2021-06-08,0
3,https://fortune.com/2021/06/08/food-supply-cha...,A new $4 billion investment from the USDA aims...,2021-06-08,0
4,https://fortune.com/2021/06/07/covid-vaccines-...,Anheuser-Busch plans biggest beer giveaway eve...,2021-06-07,0
...,...,...,...,...
2725,https://fortune.com/2019/06/20/hope-hicks-full...,Hope Hicks Interview Transcript Released: Read...,2019-06-20,0
2726,https://fortune.com/2019/06/20/trump-kim-north...,Why the Trump-Kim Nuclear Show Is Set for Act ...,2019-06-20,0
2727,https://fortune.com/2019/06/20/roger-stone-ins...,Roger Stone’s Social Media Posts Violated His ...,2019-06-20,0
2728,https://fortune.com/2019/06/20/kids-border-tex...,A Dangerous Situation Is Unfolding for Kids at...,2019-06-20,0


## Crawling from https://www.nytimes.com/

In [21]:
file_out6 = []
for y in range(2018, 2021):
  for m in range(1, 10):
    for d in range(1, 10):
      url = 'https://www.nytimes.com/sitemap/'+str(y)+'/0'+str(m)+'/0'+str(d)+'/'
      reqs = requests.get(url)
      #print(reqs)
      page_source = soup(reqs.content, 'html.parser')
      page_source = page_source.find('ul', class_ = 'css-cmbicj')
      #print(page_source)
      for line in page_source:
        title = line.find('a').get_text()
        link = line.find('a').get('href')
        time = str(y) + '-' + str(m) + '-' + str(d)
        # print(title)
        # print(link)
        # print(time)
        file_out6.append({'article_link':link, 'headline':title, 'date': time, "is_sarcastic":0})
for y in range(2018, 2021):
  for m in range(10, 13):
    for d in range(10, 28):
      url = 'https://www.nytimes.com/sitemap/'+str(y)+'/'+str(m)+'/'+str(d)+'/'
      reqs = requests.get(url)
      #print(reqs)
      page_source = soup(reqs.content, 'html.parser')
      page_source = page_source.find('ul', class_ = 'css-cmbicj')
      #print(page_source)
      for line in page_source:
        title = line.find('a').get_text()
        link = line.find('a').get('href')
        time = str(y) + '-' + str(m) + '-' + str(d)
        # print(title)
        # print(link)
        # print(time)
        file_out6.append({'article_link':link, 'headline':title, 'date': time, "is_sarcastic":0})
json_res6 = json.dumps(file_out6)
df6 = pd.read_json(StringIO(json_res6))
df6
      

,article_link,headline,date,is_sarcastic
0,https://www.nytimes.com/2017/12/31/opinion/tom...,Tom Brokaw: You Can Find the Entire World Insi...,2018-01-01,0
1,https://www.nytimes.com/2017/12/31/world/ameri...,New York Family of 5 Among 10 Americans Killed...,2018-01-01,0
2,https://www.nytimes.com/2017/12/31/nyregion/me...,The Wall of Love Outside a Jail,2018-01-01,0
3,https://www.nytimes.com/2017/12/31/opinion/new...,The Sentient-Being Diet,2018-01-01,0
4,https://www.nytimes.com/2017/12/31/opinion/men...,Changing the Script,2018-01-01,0
...,...,...,...,...
61606,https://www.nytimes.com/2020/12/27/sports/jets...,Jets Start a Winning Streak and Diminish Brown...,2020-12-27,0
61607,https://www.nytimes.com/2020/12/27/crosswords/...,Who Wants Candy?,2020-12-27,0
61608,https://www.nytimes.com/2020/12/27/business/st...,"As Bills Pile Up, Many Anxiously Keep Tabs on ...",2020-12-27,0
61609,https://www.nytimes.com/2020/12/27/sports/foot...,Ravens Snatch Back Giants’ Playoff Hopes,2020-12-27,0


In [27]:
#merge sarcasm_title_article dataframe and official_articles dataframe
news = [result,df4, df5, df6]
data = pd.concat(news)
data = data.reset_index()
data

,level_0,index,article_link,headline,date,is_sarcastic
0,0,0.0,https://babylonbee.com/'/news/new-york-times-n...,New York Times Now Claims '1619 Project' Was S...,2021-06-09,1
1,1,1.0,https://babylonbee.com/'/news/10-reasons-homes...,10 Most Devastating Impacts Of Homeschooling Y...,2021-06-09,1
2,2,2.0,https://babylonbee.com/'/news/i-do-not-study-s...,"'I Don't Study Science, I AM The Science,' Gro...",2021-06-09,1
3,3,3.0,https://babylonbee.com/'/news/half-black-man-o...,Half-Black Man Ordered To Pay Himself Reparations,2021-06-09,1
4,4,4.0,https://babylonbee.com/'/news/experts-unsure-w...,Experts Completely Baffled As To Where Joe Bid...,2021-06-09,1
...,...,...,...,...,...,...
116268,61606,NaN,https://www.nytimes.com/2020/12/27/sports/jets...,Jets Start a Winning Streak and Diminish Brown...,2020-12-27,0
116269,61607,NaN,https://www.nytimes.com/2020/12/27/crosswords/...,Who Wants Candy?,2020-12-27,0
116270,61608,NaN,https://www.nytimes.com/2020/12/27/business/st...,"As Bills Pile Up, Many Anxiously Keep Tabs on ...",2020-12-27,0
116271,61609,NaN,https://www.nytimes.com/2020/12/27/sports/foot...,Ravens Snatch Back Giants’ Playoff Hopes,2020-12-27,0


#4. Nhận xét:
<a name="nhanxet"></a>
* Thư viện bs4 hỗ trợ chúng em khá nhiều trong việc crawling data từ các websites.
* Trong lúc crawling, có những trang web denied requests của chúng em dẫn đến việc chúng em phải đổi sang trang khác. Song song, có các trang mất thời gian rất nhiều để phản hồi.
* Mật độ các tin tức chính thống nhiều hơn tin tức châm biếm nên có thể gây chênh lệnh về phân bố dữ liệu, ảnh hưởng đến quá trình train model.
